# Estimate Bernoulli draws probabilility

We estimate a simple model of ``n`` independent Bernoulli draws, with
probability ``α``. First, we load the packages we use.

In [1]:
using StatisticalRethinking
using TransformVariables
using LogDensityProblems
using DynamicHMC
using MCMCDiagnostics
using Parameters
using Statistics

Then define a structure to hold the data.
For this model, the number of draws equal to `1` is a sufficient statistic.

In [2]:
"""
Toy problem using a Bernoulli distribution.
We model `n` independent draws from a ``Bernoulli(α)`` distribution.
"""
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end

Main.##374.BernoulliProblem

Then make the type callable with the parameters *as a single argument*.  We
use decomposition in the arguments, but it could be done inside the function,
too.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    # log likelihood: the constant log(combinations(n, s)) term
    # has been dropped since it is irrelevant to sampling.
    #sum([s1 * log(α) + (n-s1) * log(1-α) for s1 in s])
    loglikelihood(Binomial(n, α), s)
end

We should test this, also, this would be a good place to benchmark and
optimize more complicated problems.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-2.6548056865833978

Recall that we need to

1. transform from ``ℝ`` to the valid parameter domain `(0,1)` for more efficient sampling, and

2. calculate the derivatives for this transformed mapping.

The helper packages `TransformVariables` and `LogDensityProblems` take care of
this. We use a flat prior (the default, omitted)

In [5]:
P = TransformedLogDensity(as((α = as𝕀,)), p)
∇P = ADgradient(:ForwardDiff, P);

Finally, we sample from the posterior. `chain` holds the chain (positions and
diagnostic information), while the second returned value is the tuned sampler
which would allow continuation of sampling.

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
0.0015 s/step ...done
MCMC, adapting ϵ (25 steps)
7.4e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0026 s/step ...done
MCMC, adapting ϵ (100 steps)
7.9e-6 s/step ...done
MCMC, adapting ϵ (200 steps)
7.8e-5 s/step ...done
MCMC, adapting ϵ (400 steps)
7.1e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
8.2e-6 s/step ...done
MCMC (1000 steps)
5.8e-6 s/step ...done


(DynamicHMC.NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([1.00779], -3.04514, 2, DoubledTurn, 0.975685, 3), NUTS_Transition{Array{Float64,1},Float64}([0.44519], -3.24392, 1, AdjacentTurn, 0.915135, 3), NUTS_Transition{Array{Float64,1},Float64}([0.141423], -3.77662, 1, DoubledTurn, 0.832607, 1), NUTS_Transition{Array{Float64,1},Float64}([0.141423], -5.09866, 1, AdjacentTurn, 0.646524, 3), NUTS_Transition{Array{Float64,1},Float64}([0.833871], -3.4606, 1, AdjacentTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([0.773915], -2.91409, 1, DoubledTurn, 0.992555, 1), NUTS_Transition{Array{Float64,1},Float64}([0.410581], -3.24484, 1, DoubledTurn, 0.896161, 1), NUTS_Transition{Array{Float64,1},Float64}([0.402524], -3.38168, 2, DoubledTurn, 0.998691, 3), NUTS_Transition{Array{Float64,1},Float64}([1.22176], -3.22842, 2, DoubledTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([1.22176], -4.66233, 1, AdjacentTurn, 0.628483, 3)  …  NUTS_Transitio

To get the posterior for ``α``, we need to use `get_position` and
then transform

In [7]:
posterior = TransformVariables.transform.(Ref(∇P.transformation), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

296.51912251589096

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.91, min/25%/median/75%/max: 0.12 0.89 0.97 1.0 1.0
  termination: AdjacentTurn => 30% DoubledTurn => 70%
  depth: 1 => 64% 2 => 36%


check the mean

In [11]:
mean(posterior_α)

0.7247017597415696

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*